In [1]:
# https://github.com/nightrome/cocostuff#downloads
# image datasets from coco 2017 train images

In [2]:
cd train2017/

/Users/mac/Project/IntelligentSearching/intelligent-searching/train2017


In [3]:
from matplotlib import pyplot as plt
import numpy as np
import cv2

In [4]:
import glob

In [5]:
image_name_list = glob.glob('*.jpg')

In [6]:
import random
im_chose = []
im_left = []
im_right = []
hist_left = []
hist_right = []
for i in range(min(len(image_name_list), 20000)): # Should be sufficiently large (>= 10k reference images to start)
    im = cv2.imread(image_name_list[i])
    if im.shape[0] > 448 and im.shape[1] > 224: # Try to find images that are large enough to have 224x224 crops from the two halves

        im_chose.append(im)
        
        left_x = random.randint(0, im.shape[0]//2 - 224)  # Crop out boundaries (rather than simply cut image into halves) to avoid simply continuity clue
        left_y = random.randint(0, im.shape[1] - 224)
        im_l = im[left_y:left_y+224, left_x:left_x+224]
        im_left.append(im_l) # y:y+h, x:x+w
        
        
        right_x = random.randint(im.shape[0]//2, im.shape[0]-224)
        right_y = random.randint(0, im.shape[1] - 224)
        im_r = im[right_y:right_y+224, right_x:right_x+224]
        im_right.append(im_r)
        

        hist_left.append(cv2.calcHist([im_l], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256]))
        hist_right.append(cv2.calcHist([im_r], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256]))        

In [7]:
import numpy as np

In [8]:
compare_method = 3 # the smaller the more accurate the match

In [9]:
hist_match = []
for i in range(len(hist_left)):
    temp = []
    for j in range(len(hist_right)):
        halfi_halfj = cv2.compareHist( hist_left[i], hist_right[j], compare_method)
        temp.append(halfi_halfj)
    matchest = np.argmin(temp)
    hist_match.append(matchest)

In [10]:
print(len(hist_match))
np.save('hist_match.npy', hist_match)

11363


In [11]:
matches = 0
for i in range(len(hist_match)):
    if hist_match[i] == i:
        matches += 1
accuracy = matches/len(hist_match) *100

In [12]:
print('accuracy:', accuracy, '%')  # 12.5%

accuracy: 12.72551262870721 %
